Модифіковуємо зображення виділяючи контури


general purpose imports

In [1]:
import os
import cv2
import torch
import ultralytics

image modifications

In [ ]:
import os
import cv2

# src_dir = './RUSSIAN_PLANES_DB/train'
# src_dir = '../RUSSIAN_PLANAE_DB/test_10/'
# src_dir = '../RUSSIAN_PLANAE_DB/original/'
# dest_dir = '../RUSSIAN_PLANAE_DB/full_modified/'

src_dir =  "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\src\\original_src"
dest_dir = "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\src\\modified_src"


for filename in os.listdir(src_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        img_path = os.path.join(src_dir, filename)
        img = cv2.imread(img_path)
        print(img)
        cv2.imshow('image', img)
        print(os.path.exists(img_path))
        #CANNY
        # edges = cv2.Canny(img,150,250)

        #LAPLACE
        # LESS_CONTRAST_EDGES (ver 1)
        # denoise = cv2.GaussianBlur(img, (3,3), 0)
        # MORE_CONTRAST_EDGES (ver 2)
        denoise = cv2.GaussianBlur(img, (1,1), 0)
        
        #FURTHER_PROCESSING
        to_gray = cv2.cvtColor(denoise,  cv2.COLOR_BGR2GRAY)
        res_lap = cv2.Laplacian(to_gray, cv2.CV_16S, ksize=3)

        cv2.imwrite(os.path.join(dest_dir, filename), res_lap)
        # cv2.imwrite(os.path.join('./test_10_images/', filename), denoise) # last arg denoise or edges
        # cv2.imwrite(os.path.join('./test_10_images/', filename), to_gray)


Для розподілення датасету по train val папках


In [ ]:
import os
import cv2
import shutil


# Шлях до папки з зображеннями
src_folder =   "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\src\\modified_src"
train_folder = "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\images\\train"
val_folder =   "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\images\\val"

# Отримати список всіх файлів у початковій папці
all_files = [f for f in os.listdir(src_folder) if os.path.isfile(os.path.join(src_folder, f))]

# Відсортувати файли (опціонально, залежить від вимог)
all_files.sort()

# Останні 50 файлів для валідаційного набору
val_files = all_files[-50:]
# Решта файлів для тренувального набору
train_files = all_files[:-50]

# Створити папки train та val, якщо вони ще не існують
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Переміщення файлів
for file_name in train_files:
    shutil.move(os.path.join(src_folder, file_name), os.path.join(train_folder, file_name))

for file_name in val_files:
    shutil.move(os.path.join(src_folder, file_name), os.path.join(val_folder, file_name))

print("Файли успішно розподілені між train та val папками.")


Тренування моделі

In [ ]:
import torch
from ultralytics import YOLO
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'avialable device: {device}')
torch.backends.cudnn.enabled = False

# Load a model
# project_folder = "C:\\Users\\Северин\\Documents\\unik\\kursova"
# model = YOLO('yolov8n.yaml')
# data = project_folder + 'src/CustomPlane_copy.yaml'

# load model
data = "CustomPlane.yaml"
model = YOLO('yolov8n.yaml')
model = YOLO('pts\\yolov8n.pt')
# cls_pw = [8.178472861085556, 104.58823529411765, 17.85140562248996, 277.8125, 50.51136363636363, 54.876543209876544, 13.7403400309119, 53.55421686746988, 13.761609907120743, 138.90625, 467.89473684210526, 467.89473684210526, 20.158730158730158, 24.092140921409214, 261.47058823529414, 25.4, 33.29588014981273, 14.11111111111111, 9.437367303609342, 8.262081784386616, 27.103658536585368, 19.755555555555556, 22.114427860696516]

# Train and use the model with 527 images
results_train = model.train(data=data, epochs=200, imgsz=1000, batch=6, augment=False, device=0) # , cls_pw=cls_pw
results_val   = model.val()
results_test  = model('..\\DB\\images\\train\\0266.png')



Тестування моделі

In [ ]:
# import ultralytics
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
model = YOLO('./pts/best_laplace.pt')

results = model.predict('../DB/images/train/0000.png')
# results = model.predict('../DB/original_images/0000.png')


for i, result in enumerate(results):
    boxes = result.boxes                # Boxes object for bounding box outputs
    masks = result.masks                # Masks object for segmentation masks outputs
    keypoints = result.keypoints        # Keypoints object for pose outputs
    probs = result.probs                # Probs object for classification outputs
    obb = result.obb                    # Oriented boxes object for OBB outputs
    # results.plot()
    result.show()                            # display to screen
    result.save(filename=f"../detections/result_{i}.jpg")  # save to disk

Рахування екземплярів класів й обчислення ваг, які дуже хотілось використати

In [ ]:
import os
from collections import defaultdict

# Шлях до папки з анотаційними файлами
annotations_dir = "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\src\\labels_all"

# Словник для збереження підрахунку об'єктів кожного класу
class_counts = defaultdict(int)

# Обхід всіх анотаційних файлів у папці
for filename in os.listdir(annotations_dir):
    if filename.endswith('.txt'):
        filepath = os.path.join(annotations_dir, filename)
        with open(filepath, 'r') as file:
            for line in file:
                class_id = int(line.split()[0])  # Отримання class_id з рядка
                class_counts[class_id] += 1

sorted_class_counts = dict(sorted(class_counts.items()))

# Знаходження загальної кількості об'єктів
total_objects = sum(class_counts.values())

# Обчислення вагових втрат для кожного класу
weighted_losses = [(total_objects / count) for _, count in sorted_class_counts.items()]

# Виведення результатів
print("Sorted class counts:", sorted_class_counts)
print("Weighted losses:", weighted_losses)


Вилучення деінде зображень й лейблів найпоширеніших класів (помилка, посварилися процеси на машині win11)

In [ ]:
import os
import shutil
import time
import portalocker
from collections import defaultdict

# Шляхи до бази даних
images_dir = "..\\DB\\images"
labels_dir = "..\\DB\\labels"
unnormalized_images_dir = "..\\DB\\unnormalized\\images"
unnormalized_labels_dir = "..\\DB\\unnormalized\\labels"

# Створення папок, якщо вони не існують
os.makedirs(unnormalized_images_dir, exist_ok=True)
os.makedirs(unnormalized_labels_dir, exist_ok=True)

# Словник з кількістю екземплярів кожного класу
class_counts = {
    0: 1087, 1: 85, 2: 498, 3: 32, 4: 176, 5: 162, 6: 647, 7: 166, 8: 646, 9: 64,
    10: 19, 11: 19, 12: 441, 13: 369, 14: 34, 15: 350, 16: 267, 17: 630, 18: 942,
    19: 1076, 20: 328, 21: 450, 22: 402
}

# Визначення порогу кількості екземплярів
threshold = 500

# Перелік найпоширеніших класів, які треба перемістити
most_common_classes = [class_id for class_id, count in class_counts.items() if count > threshold]

def copy_and_remove_file(src, dst):
    """Копіювання файлів з подальшим видаленням"""
    while True:
        try:
            with open(src, 'r+') as f:
                portalocker.lock(f, portalocker.LOCK_EX)
                shutil.copy(src, dst)
                os.remove(src)
                portalocker.unlock(f)
            break
        except (PermissionError, portalocker.exceptions.LockException):
            print(f"PermissionError: Retrying to copy and remove {src}")
            time.sleep(1)

def process_directory(data_type):
    """Обробка директорії для train або val"""
    images_subdir = os.path.join(images_dir, data_type)
    labels_subdir = os.path.join(labels_dir, data_type)
    
    for label_file in os.listdir(labels_subdir):
        if label_file.endswith('.txt'):
            label_path = os.path.join(labels_subdir, label_file)
            image_path = os.path.join(images_subdir, os.path.splitext(label_file)[0] + '.png')
            
            if os.path.exists(image_path):
                with open(label_path, 'r') as f:
                    for line in f:
                        class_id = int(line.split()[0])
                        if class_id in most_common_classes:
                            print(f"Processing file: {image_path}")
                            copy_and_remove_file(image_path, os.path.join(unnormalized_images_dir, os.path.basename(image_path)))
                            copy_and_remove_file(label_path, os.path.join(unnormalized_labels_dir, os.path.basename(label_path)))
                            break  # Переміщуємо тільки один раз для кожного файлу

# Обробка тренувальної і валідаційної директорій
process_directory('train')
process_directory('val')

print("Переміщення завершено.")


Примноження зображень й лейблів що рідкісних класів

In [ ]:
import os
import shutil
from collections import Counter

# Шлях до бази даних
images_dir = "..\\DB\\images\\train"
labels_dir = "..\\DB\\labels\\train"

# Словник з кількістю екземплярів кожного класу
class_counts = {
    0: 1087, 1: 85, 2: 498, 3: 32, 4: 176, 5: 162, 6: 647, 7: 166, 8: 646, 9: 64,
    10: 19, 11: 19, 12: 441, 13: 369, 14: 34, 15: 350, 16: 267, 17: 630, 18: 942,
    19: 1076, 20: 328, 21: 450, 22: 402
}

# Визначення порогу кількості екземплярів для збалансування
threshold = 200

# Поріг для найрідкісніших класів
rarest_threshold = 75

# Максимальна кількість копій для одного файлу
max_copies_per_file = 5

# Максимальна кількість копій для файлів з класами, кількість екземплярів яких менше за 30
max_rarest_copies = 20

# Перелік найрідкісніших класів
rarest_classes = {class_id: count for class_id, count in class_counts.items() if count < rarest_threshold}

# Перелік найменш поширених класів, які треба примножити
least_common_classes = {class_id: count for class_id, count in class_counts.items() if rarest_threshold <= count < threshold}

def calculate_copy_factor(label_path):
    """Обчислення коефіцієнта копіювання на основі кількості рідкісних класів"""
    with open(label_path, 'r') as f:
        lines = f.readlines()
        total_classes = len(lines)
        rare_count = sum(1 for line in lines if int(line.split()[0]) in rarest_classes)
    
    if rare_count / total_classes < 0.3:  # Якщо кількість рідкісних класів менше 30% від загальної кількості класів
        return 0  # Не створюємо копії
    if rare_count < 1:
        return 0
    elif rare_count < 30:
        return min(max_rarest_copies, int(max_rarest_copies / rare_count))
    else:
        return max(1, max_copies_per_file // 2)  # Зменшуємо до половини максимальної кількості копій

def duplicate_files(image_path, label_path, copy_factor, copy_counter):
    """Примноження файлів з зображенням та лейблом"""
    base_image_name = os.path.splitext(os.path.basename(image_path))[0]
    base_label_name = os.path.splitext(os.path.basename(label_path))[0]
    
    for i in range(copy_factor):
        if copy_counter[image_path] < copy_factor:
            new_image_name = f"{base_image_name}_dup_{copy_counter[image_path]}.png"
            new_label_name = f"{base_label_name}_dup_{copy_counter[image_path]}.txt"
            
            new_image_path = os.path.join(os.path.dirname(image_path), new_image_name)
            new_label_path = os.path.join(os.path.dirname(label_path), new_label_name)
            
            if os.path.exists(image_path) and os.path.exists(label_path):
                shutil.copy(image_path, new_image_path)
                shutil.copy(label_path, new_label_path)
                copy_counter[image_path] += 1
            else:
                print(f"Пропуск файлу {image_path} або {label_path}, оскільки він не існує.")

def process_directory():
    """Обробка директорії train"""
    files_to_duplicate = []

    for label_file in os.listdir(labels_dir):
        if label_file.endswith('.txt'):
            label_path = os.path.join(labels_dir, label_file)
            image_path = os.path.join(images_dir, os.path.splitext(label_file)[0] + '.png')
            
            if os.path.exists(image_path) and os.path.exists(label_path):
                copy_factor = calculate_copy_factor(label_path)
                if copy_factor > 1:
                    files_to_duplicate.append((image_path, label_path, copy_factor))
            else:
                print(f"Пропуск файлу {image_path} або {label_path}, оскільки він не існує.")
    
    copy_counter = Counter()
    
    for image_path, label_path, copy_factor in files_to_duplicate:
        duplicate_files(image_path, label_path, copy_factor, copy_counter)

# Обробка тільки тренувальної директорії
process_directory()

print("Примноження завершено.")

чищення датасету від дублікатів після примножень

In [96]:
import os
import glob

def delete_files(directory, pattern):
    # Створюємо шлях до файлів за допомогою шаблону
    search_pattern = os.path.join(directory, pattern)
    
    # Знаходимо всі файли, що відповідають шаблону
    files = glob.glob(search_pattern)
    
    # Видаляємо кожен файл
    for file in files:
        try:
            os.remove(file)
            # print(f"Файл {file} видалено.")
        except Exception as e:
            print(f"Помилка при видаленні {file}: {e}")

# Каталоги
directories = ["..\\DB\\images\\train", "..\\DB\\labels\\train"]
# Шаблон для пошуку файлів
pattern = "*_dup_*"

# Видаляємо файли в кожному каталозі
for directory in directories:
    delete_files(directory, pattern)


Знаходження зображень де є об'єкти найрідкісніших класів й дублювання

In [ ]:
import os
import shutil

# Шлях до бази даних
images_dir = "..\\DB\\images\\train"
labels_dir = "..\\DB\\labels\\train"

# Словник з кількістю екземплярів кожного класу
class_counts = {
    0: 1087, 1: 85, 2: 498, 3: 32, 4: 176, 5: 162, 6: 647, 7: 166, 8: 646, 9: 64,
    10: 19, 11: 19, 12: 441, 13: 369, 14: 34, 15: 350, 16: 267, 17: 630, 18: 942,
    19: 1076, 20: 328, 21: 450, 22: 402
}

# Поріг для найрідкісніших класів
rarest_threshold = 30

# Перелік найрідкісніших класів
rarest_classes = {class_id for class_id, count in class_counts.items() if count < rarest_threshold}

def find_images_with_rarest_classes(labels_dir, images_dir, rarest_classes):
    """Знаходить зображення з об'єктами найрідкісніших класів"""
    images_with_rarest_classes = []

    for label_file in os.listdir(labels_dir):
        if label_file.endswith('.txt'):
            label_path = os.path.join(labels_dir, label_file)
            image_path = os.path.join(images_dir, os.path.splitext(label_file)[0] + '.png')

            if not os.path.exists(image_path):
                print(f"Пропуск файлу {image_path}, оскільки він не існує.")
                continue

            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    class_id = int(line.split()[0])
                    if class_id in rarest_classes:
                        images_with_rarest_classes.append((image_path, label_path))
                        break  # Достатньо знайти хоча б один рідкісний клас

    return images_with_rarest_classes

def duplicate_files(image_path, label_path, max_copies_per_file):
    """Примноження файлів з зображенням та лейблом"""
    base_image_name = os.path.splitext(os.path.basename(image_path))[0]
    base_label_name = os.path.splitext(os.path.basename(label_path))[0]

    for i in range(max_copies_per_file):
        new_image_name = f"{base_image_name}_dup_{i}.png"
        new_label_name = f"{base_label_name}_dup_{i}.txt"

        new_image_path = os.path.join(os.path.dirname(image_path), new_image_name)
        new_label_path = os.path.join(os.path.dirname(label_path), new_label_name)

        if os.path.exists(image_path) and os.path.exists(label_path):
            shutil.copy(image_path, new_image_path)
            shutil.copy(label_path, new_label_path)
        else:
            print(f"Пропуск файлу {image_path} або {label_path}, оскільки він не існує.")

# Знаходимо зображення з найрідкіснішими класами
images_with_rarest_classes = find_images_with_rarest_classes(labels_dir, images_dir, rarest_classes)

# Встановлюємо максимальну кількість копій
max_copies_per_file = 5

# Дублюємо знайдені файли
for image_path, label_path in images_with_rarest_classes:
    duplicate_files(image_path, label_path, max_copies_per_file)

print("Примноження завершено.")


насліддя минулого розробника

In [ ]:
import cv2
# model = YOLO('D:/DesktopFolders/Універ/5курс/Сем_2/ООП/neuron/ultralytics-main/examples/best_laplace_contrast.pt')
model = YOLO('./pts/best_laplace_contrast.pt')
# image='D:/temp_2/0223.png'
image='../img_db/RUSSIAN_PLANAE_DB/0223.png'
img = cv2.imread(image)
class_name_dict = {
    0: 'Helicopter',
    1:'USSR|CIV/Transport|AN-124',
    2:'USSR|CIV/Transport/Recon.|AN-24/26/30',
    3:'USSR|Transport|AN-22',
    4:'USSR|Transport|AN-12',
    5:'USSR|CIV/Transport|AN-2',
    6:'USSR|Transport/Refuel.|IL-76/78',
    7:'USSR|Surv./MPA|IL-20/38',
    8:'CZECHOSLOVAKIA|Trainer/G-A|L-39',
    9:'RUSSIA|Trainer|YAK-130',
    10:'USSR|AWACS|A-50',
    11:'USSR|Bomber|TU-22',
    12:'USSR|Strat.Bomber|TU-22M',
    13:'USSR|Strat.Bomber/MAP|TU-95/142',
    14:'USSR|Strat.Bomber|TU-160',
    15:'USSR|Trainer|TU-134',
    16:'CIV AIRCRAFT',
    17:'USSR|Interceptor/G-A|MIG-31',
    18:'USSR|Fighter|MIG-29',
    19:'USSR|Mutlirole Fighter|SU-27/30/33/35',
    20:'USSR|Fighter/Bomber|SU-34',
    21:'USSR|CAS|SU-25',
    22:'USSR|Tact.Bomber|SU-24'
}
results = model(img)[0]   
for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(img, class_name_dict[int(class_id)].upper(), (int(x1), int(y1 - 10)),
            cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 0), 1, cv2.LINE_AA)
            cv2.imwrite('recognized.jpg',img)